# Exploring Services via the `manager-api`

---

Links to notebooks in this repository:

[Quickstart Tutorial](./quickstart_tutorial.ipynb) | [Introduction](../../../../../Downloads/00_introduction.ipynb) | [Services](01_services.ipynb) | [Sleep Staging](./02_sleep_staging.ipynb) | [Ensembling Sleep Staging](./03_ensembling_sleep_staging.ipynb) | [Sleep Dynamics](./04_sleep_dynamics.ipynb) | [Luna Toolbox Integration](./05_luna_integration.ipynb)

---

In this section, we will guide you through the various services available in SLEEPYLAND. We will not cover how to access or call the `gui` service and the `notebook` service directly, as the `gui` is already running in the background with the main purpose of easily expose the SLEEPYLAND tool, and the `notebook` is actually the interface you are currently using to interact with all other services.



> ## *Accessing the `gui` Service*
> Even though you are using the Jupyter Notebook to interact with SLEEPYLAND, you can still access the `gui` when needed. This service provides a web-based interface for interacting with the various functionalities of SLEEPYLAND.
>
> To open the GUI service, follow these steps:
> 1. Ensure that all containers are running. You can check the status of all your Docker containers by executing the following command in your terminal:
>
> ```bash
> docker ps
> ```
>
> This command lists all the running containers along with their names and status. Look for the container named gui in the output to confirm that the `gui` service is up.
>
> 2. Open your web browser and navigate to the following URL:
>```plaintext
> http://localhost:8887
> ```
>
> This will take you to the SLEEPYLAND GUI, where you can perform various tasks interactively.
>
> By using the Jupyter Notebook, you can efficiently manage and interact with other services while the GUI runs in the background for additional functionalities.






> ## *Helper function for making POST requests*
> Below a reusable helper function for making POST requests and performing specific tasks. The function defined here allow you to interact with the different services via the `manager-api` for different operations such as downloading data, harmonizing datasets, retrieving channels, making predictions.

> Important Note: Make sure all Docker containers are running before you attempt to send requests to the `manager-api`.

In [ ]:
import requests

# Define the base URL for the manager-api
MANAGER_API_BASE_URL = "http://manager-api:8989"

def make_post_request(endpoint, data=None, params=None):
    """
    Helper function to make a POST request to the specified endpoint.

    Parameters:
        endpoint (str): The API endpoint to hit.
        data (dict, optional): The form data to send in the request.
        params (dict, optional): The URL parameters to send in the request.

    Returns:
        dict: The JSON response if the request is successful.
    """
    url = f"{MANAGER_API_BASE_URL}/{endpoint}"
    response = requests.post(url, data=data, params=params)
    if response.status_code == 200:
        print("Success:", response.json())
        return response.json()
    else:
        print(f"Failed with status code {response.status_code}")
        return None

## Accessing the `nsrr-download` Service

---

The `nsrr-download` service is responsible for downloading sleep data from the National Sleep Research Resource (NSRR). It interacts with the underlying file system to manage the input files, execute download commands, and handle dataset organization. 

In this section, we will explain how to access and use the `nsrr-download` service via the `manager-api`. 

The `nsrr-download` service exposes the following endpoint:

- **`POST /download_data`**: This endpoint allows you to download data by providing a token and a list of selected recordings/data.

### Key Functionalities of the `nsrr-download` Service

1. **Download Data**:
   - The service accepts a list of recordings/data in the format `edf_path+ann_path`, where each data consists of an EDF file and its corresponding XML annotation file.
   - It uses the `download_data` function to initiate the download process for each specified data.

2. **Command Execution**:
   - The service runs a shell script (`nsrr_download.sh`) to handle the actual downloading of files.
   - It utilizes the `subprocess` module to execute shell commands securely.

3. **File Management**:
   - After downloading, the service creates a folder into the `input` volume, and the files are moved into this new folder.

4. **Error Handling**:
   - The service provides error handling to ensure that any issues encountered during the download process are logged and communicated back to the user.

To interact with the `nsrr-download` service, you'll use the `manager-api`, which handles requests and forwards them to the appropriate service. Here’s how you can do it:

In [ ]:
# This function sends a request to download specific datasets (EDF and annotation files) from the manager-api.
# It uses the user's token for authentication and specifies the paths of the datasets to be downloaded.
def download_data(edf_path, annotation_path, token):
    # Combine the edf and annotation paths
    data_path = edf_path + "+" + annotation_path
    
    # Specify the datasets to download
    datasets = [data_path]
    
    data = {'token': token, 'selected_datasets': datasets}
    make_post_request("download_data", data=data)

In [ ]:
# Replace 'your_token_here' with your actual NSRR token
token = 'your_token_here'

# Specify the edf and annotation paths for the data you want to download
edf_path = "abc/polysomnography/edfs/baseline/abc-baseline-900001.edf"
annotation_path = "abc/polysomnography/annotations-events-nsrr/baseline/abc-baseline-900001-nsrr.xml"

# Use the download_data function to download the specified EDF and annotation files.
download_data(edf_path, annotation_path, token)

## Accessing the `wild-to-fancy` service

---

The `wild-to-fancy` service is responsible for harmonizing PSG datasets, converting them into a format suitable for analysis. This service interacts with uploaded datasets and processes them according to specified commands.

In this section, we will explain how to access and use the `wild-to-fancy` service via the `manager-api`.

The `wild-to-fancy` service exposes the following endpoint:

- **`POST /harmonize`**: This endpoint allows you to harmonize uploaded datasets by providing the folder name and dataset type.

### Key Functionalities of the `wild-to-fancy` Service

1. **Harmonize Datasets**:
   - The service processes the uploaded datasets, specifically `.edf` and annotation files, such as `.xml`, according to predefined command templates specified in JSON configuration files.
   - It organizes the harmonized output files in the appropriate directory structure.

2. **Command Execution**:
   - The service runs specific shell commands to perform the harmonization process, ensuring that each command is executed in the correct context.

3. **File Management**:
   - After processing, the service cleans up temporary directories and removes the original uploaded files to maintain a tidy workspace.

4. **Error Handling**:
   - The service provides error handling to ensure that any issues encountered during the harmonization process are logged and communicated back to the user.



To interact with the `wild-to-fancy` (i.e., harmonizer) service, you'll use the `manager-api` to send a request to harmonize the datasets. Here’s how to do it:

> Before sending a request, ensure that your dataset is uploaded and structured properly (check the `input` folder). The uploaded dataset should include the necessary `.edf` and annotation files.

In [ ]:
# This function sends a request to harmonize the specified dataset, which ensures that the dataset conforms to a standard format.
def harmonize_data(dataset):
    data = {'dataset': dataset}
    make_post_request("harmonize", data=data)

In [ ]:
dataset = 'abc' 

# Use the harmonize_data function to harmonize the dataset for consistency.
harmonize_data(dataset)

## Accessing the `usleepyland` service

---
The `usleepyland` service is responsible for processing and generating evaluations/predictions based on uploaded PSG datasets. It is designed to operate on uploaded datasets, which are processed based on specified commands and configurations.

In this section, we will explain how to access and use the usleepyland service via the manager-api.

The `usleepyland` service exposes the following endpoints:

- **`GET /get_channels`**: This endpoint allows you to retrieve the channel information for each dataset.
- **`POST /auto_evaluate`**: This endpoint allows you to first harmonize uploaded datasets via the `POST /harmonize` endpoint, then evaluate the harmonized PSG files (i.e., generating sleep stage predictions and evaluating the models performance).
- **`POST /evaluate`**: This endpoint allows you to directly evaluate already harmonized (`.h5` and `.ids`) data.
- **`POST /predict_one`**: This endpoint allows you to generate sleep stage predictions from one `.edf` file uploaded by the user.

### Key Functionalities of the `usleepyland` Service

1. **Retrieve Channel Information**:
   - The service provides information about the channels available in the uploaded datasets.
   - It helps users understand the data structure and select the appropriate channels for prediction.
2. **Evaluate/Predict**:
   - The evaluate service allows you to asses the performance of the selected model, comparing its prediction against the provided annotations.
   - The predict service allows you to asses the performance of the selected model odirectly on the edf file uploaded by the user.



In [ ]:
# This function sends a request to retrieve the available channels for a given dataset.
# Channels could include EEG AND/OR EOG AND/OR EMG derivations depending on the dataset.
def get_channels(dataset):
    params = {'dataset': dataset}
    return make_post_request("get_channels", params=params)

In [ ]:
# Use the get_channels function to retrieve the available channels for the dataset.
response = get_channels(dataset)

eeg_channels = response["eeg_channels"]
eog_channels = response["eog_channels"]
emg_channels = ['']

After selecting the channels for evaluation/prediction, you can send a request to the `usleepyland` service to automatically generate the sleep stages. Use the following code snippet to initiate the `evaluate` process (make this post request only if you are already harmonize the data - refer to the `wild-to-fancy` service above).

Explanation of the parameters:
- `folder_root_name`: The root folder name where the input files are stored.
- `output_folder_name`: The folder name for the output files.
- `eeg_channels`: A list of EEG channels to use for prediction.
- `eog_channels`: A list of EOG channels to use for prediction.
- `emg_channels`: A list of EMG channels to use for prediction.
- `dataset`: The name of the NSRR dataset to predict.
- `models`: A list of feature-based and/or deep learning-based models.

> In the [Sleep Staging](./02_sleep_staging.ipynb) notebook we will properly show you how to interact with the output/metrics returned by the `evaluate_data` function.

In [ ]:
# This function sends a request to perform a prediction task using the specified EEG, EOG, and EMG channels for the given dataset.
# The results are stored in the specified output folder.
def evaluate_data(folder_root_name, output_folder_name, eeg_channels, eog_channels, emg_channels, dataset, models):
    data = {
        'folder_root_name': folder_root_name,
        'folder_name': output_folder_name,
        'eeg_channels': eeg_channels,
        'eog_channels': eog_channels,
        'emg_channels': emg_channels,
        'dataset': dataset,
        'models': models
    }
    make_post_request("evaluate", data=data)

In [ ]:
# Define the channels and dataset details for the sleep staging prediction

models = ['yasa,usleep']

# Use the predict_data function to run the prediction task on the dataset using specified channels.
evaluate_data(dataset, 'output_abc', eeg_channels, eog_channels, emg_channels, dataset, models)

## Accessing multiple services at once

Below we demonstrate how to exploit the `manager-api` by calling two distinct services — `wild-to-fancy` and `usleepyland` — in a single command. The `auto_evaluate_data` function simplifies the process by combining these steps into one streamlined command. By using `auto_evaluate_data`, you can harmonize the dataset to ensure it meets the required formats and then immediately perform predictions on the same dataset.

> Before exploiting the `auto_evaluate_data` - if you already run the pipiline above - you should first clean up the `input` volume or exploit a different NSRR dataset.

In [ ]:
# Run this cell to download a different NSRR dataset.
# Replace 'your_token_here' with your actual NSRR token
token = 'your_token_here'

# Specify the edf and annotation paths for the data you want to download
edf_path = "shhs/polysomnography/edfs/shhs1/shhs1-200001.edf"
annotation_path = "shhs/polysomnography/annotations-events-nsrr/shhs1/shhs1-200001-nsrr.xml"

# Use the download_data function to download the specified EDF and annotation files.
download_data(edf_path, annotation_path, token)

In [ ]:
# Use the get_channels function to retrieve the available channels for the dataset.
dataset = 'shhs'
response = get_channels(dataset)

eeg_channels = response["eeg_channels"]
eog_channels = response["eog_channels"]
emg_channels = ['']

In [ ]:
def auto_evaluate_data(folder_root_name, output_folder_name, eeg_channels, eog_channels, emg_channels, dataset, models):
    data = {
        'folder_root_name': folder_root_name,
        'folder_name': output_folder_name,
        'eeg_channels': eeg_channels,
        'eog_channels': eog_channels,
        'emg_channels': emg_channels,
        'dataset': dataset,
        'models': models
    }
    make_post_request("auto_evaluate", data=data)

In [ ]:
# Define the channels and dataset details for the sleep staging prediction

models = ['yasa,usleep']

# Auto Evaluate Data
# Use the auto_evaluate_data function to perform both harmonization and prediction in one step.
auto_evaluate_data(dataset, 'output_shhs', eeg_channels, eog_channels, emg_channels, dataset, models)